<a href="https://colab.research.google.com/github/lucascheng24/comp4434BigData_Fake_and_real_news_dataset/blob/main/Fake_and_real_news_dataset/classifiers/RandomForest/randomForest_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Random Forests: 0.64

In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn import metrics
from sklearn.metrics import f1_score
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


In [15]:
from google.colab import drive
# drive.mount('/content/drive')
data = pd.read_csv('/content/drive/MyDrive/comp4434/preprocess_data_set/Fake_and_real_news_dataset/newsData_ratio7_3.csv')

In [16]:
data.shape

(30596, 251)

In [2]:
dataPath = '../../preprocess_data_set/newsData_ratio7_3.csv'

df_train = pd.read_csv(dataPath)

# df_train = data
df_train.shape

(30596, 251)

In [3]:
# randomly shuffle the rows
df_train = df_train.sample(frac=1, random_state=42)

df_train = df_train.reset_index()
df_train = df_train.drop('index', axis=1)
df_train

,0,1,2,3,4,5,6,7,8,9,...,241,242,243,244,245,246,247,248,249,target
0,24.415960,-6.542249,23.304224,-73.579720,76.860611,65.102158,-13.393353,61.607204,-68.356812,30.199667,...,-86.527779,54.444340,78.129257,-87.431534,83.768745,112.784492,-51.913998,56.588608,-131.092789,0
1,14.416313,117.129875,-67.318680,-138.319870,-72.704147,119.988319,26.551735,145.345779,-154.938400,-188.729782,...,-136.380554,115.892647,103.903809,-130.659760,122.598938,259.467499,-6.670557,135.923965,-36.484703,0
2,-9.635453,16.386848,-157.652969,-80.735062,142.014816,-29.194136,37.299801,116.060081,-100.853287,-14.261718,...,-182.015640,70.160660,56.286911,-169.687637,89.847687,273.684418,-141.888168,79.751862,-103.945938,0
3,76.891594,102.174530,4.846596,-261.197784,-85.319939,137.473740,-70.686531,32.051090,-6.491718,-1.925670,...,-85.190681,79.894341,103.243690,-54.490532,105.887230,234.383926,-131.260696,63.626381,-98.301445,0
4,26.785667,25.261181,-7.442734,-7.134859,10.852419,18.031885,9.479840,4.042623,-6.164218,13.578835,...,-22.320877,1.298880,-9.595810,-1.912570,-15.478306,18.723402,-11.191423,2.858147,-6.210856,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30591,-1.136871,33.806595,53.596951,-14.134282,-55.039295,19.461432,-62.288479,40.847351,-106.527542,23.919147,...,-9.444671,49.586140,59.054657,-12.712838,48.038528,89.623146,-40.107193,95.260452,-62.034592,1
30592,-0.284165,25.422646,2.497849,-8.947376,-36.106369,29.048079,-8.172162,28.610073,-23.188131,-27.730436,...,4.322698,22.755190,8.053110,2.871661,8.147329,12.612817,-6.104894,10.478649,2.183957,0
30593,71.339561,16.005495,51.056477,-179.901749,-14.259531,126.382141,-4.463017,42.901329,53.434349,-14.907699,...,-106.613243,-39.113091,48.542309,-67.660194,58.561333,206.557541,-90.538963,154.817764,-92.831696,0
30594,81.606827,-113.467224,-3.810923,-238.568436,70.856094,87.432976,53.989868,111.472473,-73.990349,42.114433,...,-40.093098,107.691345,-5.296010,-85.728683,78.135361,180.972107,-158.033691,8.803612,-97.057518,0


In [4]:
train_y = df_train['target']
train_x = df_train.drop(columns=['target'])

In [5]:
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(train_x, train_y, test_size=0.3,random_state=109) # 70% training and 30% test

In [21]:
robust_scaler = RobustScaler().fit(X_train.values)
trans_x = robust_scaler.transform(X_train.values)

In [22]:
trans_x.shape

(21417, 250)

In [7]:
# Create a pipeline with PCA and D-forests
# higher principle components and estimators increase the accuracy as well as the model construction time
# currently it takes 2 minutes to construct the model 
pc_num = 100

pipe = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', RandomForestClassifier(n_estimators = 300))
])

pipe.fit(X_train, y_train)

Pipeline(steps=[('pca', PCA(n_components=100)),
                ('clf', RandomForestClassifier(n_estimators=300))])

In [8]:
y_pred = pipe.predict(X_test)

In [9]:
print(metrics.classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       1.00      0.94      0.97      6811
           1       0.85      0.99      0.91      2368

    accuracy                           0.95      9179
   macro avg       0.92      0.96      0.94      9179
weighted avg       0.96      0.95      0.95      9179



In [26]:
from sklearn.model_selection import KFold, cross_val_score

k_folds = KFold(n_splits = 5)

pc_num = 100

pipe_cv = Pipeline([
    ('pca', PCA(n_components = pc_num)), # choose number of components to keep
    ('clf', RandomForestClassifier(n_estimators = 300))
])

scores = cross_val_score(pipe_cv, train_x, train_y, cv = k_folds)

print("Cross Validation Scores: ", scores)
print("Average CV Score: ", scores.mean())

Cross Validation Scores:  [0.9498366  0.94950155 0.9472136  0.95701912 0.94999183]
Average CV Score:  0.950712540068596


Try different PC and analysis the performance

In [28]:
from sklearn.model_selection import KFold, cross_val_score


accuracy_list = []


k_folds = KFold(n_splits = 5)

# pc_num = 1
#  it runs for 2 hours with 10 results
for i in range(250, 0, -1):
    pipe_cv = Pipeline([
        ('pca', PCA(n_components = i)), # choose number of components to keep
        ('clf', RandomForestClassifier(n_estimators = 300))
    ])
    
    score = cross_val_score(pipe_cv, train_x, train_y, cv = k_folds).mean()
    accuracy_list.append(score)
    print("Average CV with ", i, " principal components: ", score)



Average CV with  250  principal components:  0.9374754461353098
Average CV with  249  principal components:  0.9380636974515252
Average CV with  248  principal components:  0.9375081579180673
Average CV with  247  principal components:  0.9379002880773163
Average CV with  246  principal components:  0.9374754354539114
Average CV with  245  principal components:  0.9375735334172891
Average CV with  244  principal components:  0.9389135682600109
Average CV with  243  principal components:  0.9378677151527386
Average CV with  242  principal components:  0.9379984074034908
Average CV with  241  principal components:  0.9381618007556021


KeyboardInterrupt: ignored

In [ ]:
# Create a scree plot
plt.plot(np.arange(1, 251), accuracy_list, 'bo-')
plt.xlabel('Number of PCs')
plt.ylabel('5-fold CV score')
plt.title('Performance under different PCs')
plt.show()